# Préparation des données
1. Etape 1 : Collecte des données
2. Etape 2 : Evaluation des données
3. Etape 3 : Netoyage des données
4. Etape 4 : Analyse et Visualisation

## Etape 1 : Collecte des données
1. Les archives Twitter de WeRateDogs
2. Les prédictions de l’image tweet
3. Données supplémentaires de l’API Twitter

In [1]:
# Chargement des librairies necessaire 
import pandas as pd
import requests
import tweepy
import json
from timeit import default_timer as timer

### 1. Les Archives Twitter de WeRateDogs  
Dans cette partie, il s'agit de télécharger manuellement le dataset `twitter-archive-enhanced.csv`.  
Cette prémière étape de consiste à mettre en evidence la collecte manuelle des données. Ces données sont nécessaire pour la suite du projet. 

In [2]:
# Chargement du ficher twitter-archive-enhanced.csv avec pandas
df_twitter = pd.read_csv("twitter-archive-enhanced.csv")

In [3]:
df_twitter.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


## 2. Les prédictions de l’image tweet  

`image-predictions.tsv` contenant les prédictions de l'image tweet est hébergé sur le serveur d'Udacity à l'addresse `https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv`.  
Notre objectif dans cette partis est de télécharger ce fichier en utilisant le module requests de python. Ce fichier sera ensuite enregistrer en locale sous le même non.   

In [4]:
# Chargement du fichier image_predictions.tsv hébergé sur le serveur d'Udacity
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
req = requests.get(url)

# Sauvegarde du fichier en local sous le nom de image-predictions.tsv
with open("image-predictions.tsv", mode="wb") as file:
    file.write(req.content)

In [5]:
# Chargement du dataset image-predictions.tsv
df_images = pd.read_csv("image-predictions.tsv", sep="\t")

In [6]:
df_images.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


## 3. Données supplémentaires de l’API Twitter
Dans cette partie, le but recherché est de collecter les données de WeRateDogs sur Tweeter. Pour ce faire, nous devons tout d'abord créer un compte developpeur aprés avoir créer un compte tweeter. Une fois ce compte créer, nous devons créer un projet dans lequel nous créons notre prémière application. C'est cette application qui va nous permettre d'acceder aux données en utilisant l'Api de tweeter(Tweepy). cependans à default de ne pas pouvoir collecter des données via l'API de tweeter, Udacy nous offres ces données collectées. Et la suite de notre travail consiste à construire nos dataframe avec ces données.£

In [7]:
df_tweet = pd.DataFrame()
with open('tweet-json.txt') as json_data:
    while True:
        content = json_data.readline()
        if content == "": break
        data_line = json.loads(content)
        data = {
            "id" : data_line["id"],
            "created_at" : data_line["created_at"],
             "user_followers" : data_line["user"]["favourites_count"],
            "user_friends" : data_line["user"]["friends_count"],
            "retweet" : data_line["retweet_count"],
            "favorite" : data_line["favorite_count"],
            "full_text" : data_line["full_text"],
        }
       
        df_tweet = pd.concat([df_tweet, pd.DataFrame([data])])

In [8]:
df_tweet

,id,created_at,user_followers,user_friends,retweet,favorite,full_text
0,892420643555336193,Tue Aug 01 16:23:56 +0000 2017,114031,104,8853,39467,This is Phineas. He's a mystical boy. Only eve...
0,892177421306343426,Tue Aug 01 00:17:27 +0000 2017,114031,104,6514,33819,This is Tilly. She's just checking pup on you....
0,891815181378084864,Mon Jul 31 00:18:03 +0000 2017,114031,104,4328,25461,This is Archie. He is a rare Norwegian Pouncin...
0,891689557279858688,Sun Jul 30 15:58:51 +0000 2017,114031,104,8964,42908,This is Darla. She commenced a snooze mid meal...
0,891327558926688256,Sat Jul 29 16:00:24 +0000 2017,114031,104,9774,41048,This is Franklin. He would like you to stop ca...
...,...,...,...,...,...,...,...
0,666049248165822465,Mon Nov 16 00:24:50 +0000 2015,114031,104,41,111,Here we have a 1949 1st generation vulpix. Enj...
0,666044226329800704,Mon Nov 16 00:04:52 +0000 2015,114031,104,147,311,This is a purebred Piers Morgan. Loves to Netf...
0,666033412701032449,Sun Nov 15 23:21:54 +0000 2015,114031,104,47,128,Here is a very happy pup. Big fan of well-main...
0,666029285002620928,Sun Nov 15 23:05:30 +0000 2015,114031,104,48,132,This is a western brown Mitsubishi terrier. Up...


## Etape 2 : Evaluations des données  

L'evaluation des données est l'étape situé entre la collect(ressemblement) et le nettoyage des données. En effet, c'est le précurseur du nettoyage. La réussite du nettoyage dependant de cette étape. L'évaluation consiste à evaluer les données afin de detecter les données sales et les données desordonnées. Elle peut se faire de deux manière differentes. Nous avons l'évaluation visuelle qui consiste à defiler l'ensemble de dataset afin de réperer les éventuelles problèmes. l'évaluation programmatique qui consiste à utiliser des outils comme pandas de python en vue de detecter les problèmes cités plus haut. La seconde méthode bien que récommander, il est possible de combiner les deux car necessaire et indispensable.  
### Etape d'évaluation des données
On distingue principalement deux étapes necessaires dans l'évaluation des données notamment la detection et la documentation.
#### 1. La detection  
Elle consiste à detecter le problème que pose nos données. Nous parlons ici des problèmes de qualité et de structure.  
#### 2. La Documentation  
Elle consiste à documenter les differentes problèmes rencontrés. Cette documentation est nécessaire pour l'évolutibilité et la reproductibilité de notre étude. Dans cette étape, il ne s'agit pas de speficier comment nous allons résoudre ce problème car c'est l'objet de l'étape de nettoyage. 

### Les problèmes des données
#### 1. les Problèmes de Qualité  
Les données présentant des problèmes de qualités sont des données dans lesquelles nous avons des données manquantes, dupliquées, incorrectes. On les appelles les données sales ou des données de faible qualité.  
#### 2. les problèmes de structure  
Nous pouvons être confronté à des données où les données ne sont pas ordonnées (désordées). Alors il convient de les corrigées pour une meuilleur expérience d'analyse. On les appelles des données à problème structurels.

### Les dimensions de la qualité des données  
Elles sont d'une importance capitale dans le processus de l'evaluation des données. En effet, les dimensions guident nos reflexions tout au long de l'evaluation et même du netoyage.  On distingues principalement 4 dimensions notamment la complétude, la validité, l'exactitude, cohérence. ces aspects sont cités de manière decroissante. Par consequent, la complétude est beaucoup plus important puis la validité et l'exactitude et enfin la cohérence.  

#### 1. Complétude(exhaustivité)  
La complétude consiste à évaluer les données afin d'identifier les données manquantes dans les lignes, les cellules et même les colonnes.

#### 2. Validité  
Aprés avoir évalué l'exhaustivité des données, il est necessaire de s'interroger sur la validité de ces données. En effet, on definie une donnée de valide s'il respecte(conforme) à un schéma definie, c'est à dire un ensemble de régle pouvant être des contraintes du monde réel ou/et des contraintes liées aux tables.  

#### 3. Exactitude
Les données inexacte ou incorrectes sont des données valide mais qui ne sont pas correctes.  
#### 4. Cohérence
Les données peuvent être valide, exacte mais incohérentes.

In [9]:
df_tweet

,id,created_at,user_followers,user_friends,retweet,favorite,full_text
0,892420643555336193,Tue Aug 01 16:23:56 +0000 2017,114031,104,8853,39467,This is Phineas. He's a mystical boy. Only eve...
0,892177421306343426,Tue Aug 01 00:17:27 +0000 2017,114031,104,6514,33819,This is Tilly. She's just checking pup on you....
0,891815181378084864,Mon Jul 31 00:18:03 +0000 2017,114031,104,4328,25461,This is Archie. He is a rare Norwegian Pouncin...
0,891689557279858688,Sun Jul 30 15:58:51 +0000 2017,114031,104,8964,42908,This is Darla. She commenced a snooze mid meal...
0,891327558926688256,Sat Jul 29 16:00:24 +0000 2017,114031,104,9774,41048,This is Franklin. He would like you to stop ca...
...,...,...,...,...,...,...,...
0,666049248165822465,Mon Nov 16 00:24:50 +0000 2015,114031,104,41,111,Here we have a 1949 1st generation vulpix. Enj...
0,666044226329800704,Mon Nov 16 00:04:52 +0000 2015,114031,104,147,311,This is a purebred Piers Morgan. Loves to Netf...
0,666033412701032449,Sun Nov 15 23:21:54 +0000 2015,114031,104,47,128,Here is a very happy pup. Big fan of well-main...
0,666029285002620928,Sun Nov 15 23:05:30 +0000 2015,114031,104,48,132,This is a western brown Mitsubishi terrier. Up...


In [10]:
df_images

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2070,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
2073,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True


In [11]:
df_twitter

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None


Pour répondre aux spécifications, les questions suivantes doivent être évaluées.

* Vous voulez seulement les évaluations originales (pas de retweets) qui ont des images. Bien qu’il y ait plus de 5 000 tweets dans l’ensemble de données, tous ne sont pas des évaluations de chiens et certains sont des retweets.

* L’évaluation et le nettoyage complets de l’ensemble de données nécessiteraient beaucoup de temps et ne sont pas nécessaires pour pratiquer et démontrer vos compétences en matière de traitement de données. Par conséquent, les exigences de ce projet consistent uniquement à évaluer et à nettoyer au moins huit (8) problèmes de qualité et au moins deux (2) problèmes de rangement dans cet ensemble de données.
* Le fait que les numérateurs de notation soient plus grands que les dénominateurs n’a pas besoin d’être nettoyé. Ce système de notation unique est une grande partie de la popularité de WeRateDogs.

Vous n’avez pas besoin de recueillir les tweets au-delà du 1er août 2017. Vous pouvez, mais notez que vous ne serez pas en mesure de recueillir les prédictions d’image pour ces tweets puisque vous n’avez pas accès à l’algorithme utilisé